# Datatilbredning for sykehus data

In [105]:
import pandas as pd

# Lese inn data og se på de første radene.
demographic_df = pd.read_csv('raw_data\demographic.csv')
hospital_df = pd.read_csv('raw_data\hospital.csv')
physiological_df = pd.read_csv('raw_data\physiological.txt', sep='\t')
severity_df = pd.read_json('raw_data\severity.json')

# Sjekke hvordan dataen ser ut i dataframe.
demographic_df.head(1)

,pasient_id,alder,kjønn,utdanning,inntekt,etnisitet
0,2,60.33899,female,12.0,$11-$25k,white


In [106]:
hospital_df.head(1)

,pasient_id,sykehusdød,oppholdslengde
0,2,1,4


In [107]:
physiological_df.head(1)

,pasient_id,blodtrykk,hvite_blodlegemer,hjertefrekvens,respirasjonsfrekvens,kroppstemperatur,lungefunksjon,serumalbumin,bilirubin,kreatinin,natrium,blod_ph,glukose,blodurea_nitrogen,urinmengde
0,2,43.0,17.097656,112.0,34.0,34.59375,98.0,NaN,NaN,5.5,132.0,7.25,NaN,NaN,NaN


In [108]:
severity_df.head(1)

,sykdomskategori_id,sykdomskategori,pasient_id,dødsfall,sykdom_underkategori,antall_komorbiditeter,koma_score,adl_pasient,adl_stedfortreder,fysiologisk_score,apache_fysiologisk_score,overlevelsesestimat_2mnd,overlevelsesestimat_6mnd,diabetes,demens,kreft,lege_overlevelsesestimat_2mnd,lege_overlevelsesestimat_6mnd,dnr_status,dnr_dag
0,A1s,ARF/MOSF,"[5, 15, 18, 23, 28, 34, 39, 43, 46, 47, 48, 58...","[0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, ...","[ARF/MOSF w/Sepsis, ARF/MOSF w/Sepsis, ARF/MOS...","[1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 2, 0, 3, 1, 3, ...","[26.0, 26.0, 26.0, 0.0, 26.0, 37.0, 0.0, 0.0, ...","[None, None, None, None, None, None, None, Non...","[2.0, None, 0.0, 5.0, 2.0, None, 0.0, None, No...","[23.5, 30.5, 40.296875, 31.6992188, 46.796875,...","[30.0, 39.0, 58.0, 42.0, 85.0, 49.0, 5.0, 76.0...","[0.6348876950000001, 0.590942383, 0.2129821780...","[0.5329589840000001, 0.481994629, 0.1169891360...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[no, no, no, no, yes, no, no, no, no, no, yes,...","[0.899999619, 0.899999619, 0.09999996400000001...","[0.9, 0.9, 0.001, 0.5, 0.000125, 0.60000000000...","[None, None, None, None, None, None, None, Non...","[None, None, None, None, None, None, None, Non..."


In [109]:
# Siden severity.df er på annen måte satt opp, må fikse det med explode listene.
exp = [col for col in severity_df.columns if severity_df[col].apply(lambda x: isinstance(x, list)).any()]
severity_df = severity_df.explode(exp)

In [110]:
# Merge dataframe
combined_df = pd.merge(demographic_df, hospital_df, on='pasient_id', how='left')
combined_df = pd.merge(combined_df, physiological_df, on='pasient_id', how='left')
combined_df = pd.merge(combined_df, severity_df, on='pasient_id', how='left')

# Fjerne duplicates basert på 'pasient_id' som kan skje i mergen.
combined_df = combined_df.drop_duplicates(subset=['pasient_id'], keep='first')
combined_df.head(5)


,pasient_id,alder,kjønn,utdanning,inntekt,etnisitet,sykehusdød,oppholdslengde,blodtrykk,hvite_blodlegemer,...,apache_fysiologisk_score,overlevelsesestimat_2mnd,overlevelsesestimat_6mnd,diabetes,demens,kreft,lege_overlevelsesestimat_2mnd,lege_overlevelsesestimat_6mnd,dnr_status,dnr_dag
0,2,60.33899,female,12.0,$11-$25k,white,1,4,43.0,17.097656,...,74.0,0.001,0.0,0,0,no,0.0,0.0,None,None
1,3,52.74698,female,12.0,under $11k,white,0,17,70.0,8.500000,...,45.0,0.790894,0.664917,0,0,no,0.75,0.5,None,None
2,4,42.38498,female,11.0,under $11k,white,0,3,75.0,9.099609,...,19.0,0.698975,0.411987,0,0,metastatic,0.9,0.5,None,None
3,5,79.88495,female,NaN,NaN,white,0,-99,59.0,13.500000,...,30.0,0.634888,0.532959,0,0,no,0.9,0.9,None,None
4,6,93.01599,male,14.0,NaN,white,1,4,110.0,10.398438,...,27.0,0.284973,0.214996,0,0,no,0.0,0.0,None,None


# Sjekke for dårlig data

In [111]:
combined_df.describe()

,alder,utdanning,sykehusdød,oppholdslengde,blodtrykk,hvite_blodlegemer,hjertefrekvens,respirasjonsfrekvens,kroppstemperatur,lungefunksjon,serumalbumin,bilirubin,kreatinin,natrium,blod_ph,glukose,blodurea_nitrogen,urinmengde
count,7740.000000,6359.000000,7740.000000,7740.000000,7740.000000,7565.000000,7740.000000,7740.000000,7740.000000,5796.000000,4891.000000,5544.000000,7683.000000,7740.000000,5828.000000,3917.000000,4048.000000,3627.000000
mean,62.712258,11.753106,0.263178,17.728682,84.412468,12.302026,97.283940,23.348450,37.116579,239.608379,2.952622,2.565599,1.790278,137.574160,7.415439,160.031146,32.582040,2185.398372
std,15.688039,3.458930,0.440387,21.995401,27.713746,9.108076,31.499505,9.549561,1.257278,109.942808,0.889463,5.318276,1.711926,6.029887,0.080844,87.828819,26.826593,1458.900940
min,-1.000000,0.000000,0.000000,-99.000000,0.000000,0.000000,0.000000,0.000000,31.699220,12.000000,0.399963,0.099991,0.099991,110.000000,6.829102,0.000000,1.000000,0.000000
25%,52.921740,10.000000,0.000000,6.000000,63.000000,6.899414,72.000000,18.000000,36.195310,155.093750,2.399902,0.500000,0.899902,134.000000,7.379883,103.000000,14.000000,1175.000000
50%,65.008000,12.000000,0.000000,11.000000,77.000000,10.599609,100.000000,24.000000,36.695310,223.984375,2.899902,0.899902,1.199951,137.000000,7.419922,135.000000,23.000000,1963.000000
75%,74.133747,14.000000,1.000000,20.000000,107.000000,15.298828,120.000000,28.000000,38.195310,304.750000,3.599609,1.899902,1.899902,141.000000,7.469727,190.000000,43.000000,2980.000000
max,101.847960,31.000000,1.000000,343.000000,195.000000,128.781250,232.000000,90.000000,41.695310,869.375000,29.000000,63.000000,21.500000,181.000000,7.769531,1092.000000,192.000000,9000.000000


In [112]:
# Fjerne rader med negativt verdi
# Ser det er i disse
relevant_cols = ['alder', 'oppholdslengde']
combined_df[relevant_cols] = combined_df[relevant_cols].query('oppholdslengde >= 0')
combined_df[relevant_cols] = combined_df[relevant_cols].query('alder >= 0')
combined_df = combined_df.dropna(subset=relevant_cols)
combined_df.describe()

,alder,utdanning,sykehusdød,oppholdslengde,blodtrykk,hvite_blodlegemer,hjertefrekvens,respirasjonsfrekvens,kroppstemperatur,lungefunksjon,serumalbumin,bilirubin,kreatinin,natrium,blod_ph,glukose,blodurea_nitrogen,urinmengde
count,7729.000000,6351.000000,7729.000000,7729.000000,7729.000000,7554.000000,7729.000000,7729.000000,7729.000000,5788.000000,4886.000000,5539.000000,7673.000000,7729.000000,5820.000000,3916.000000,4047.000000,3626.000000
mean,62.750226,11.752322,0.263165,17.813042,84.428581,12.303301,97.248764,23.340018,37.115028,239.691335,2.952798,2.564954,1.790356,137.574848,7.415406,160.021195,32.574030,2185.793407
std,15.611111,3.460267,0.440380,21.739318,27.723064,9.111883,31.486704,9.547923,1.257040,109.914772,0.889258,5.319370,1.711545,6.031115,0.080855,87.837826,26.825066,1458.908132
min,18.041990,0.000000,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000,31.699220,12.000000,0.399963,0.099991,0.099991,110.000000,6.829102,0.000000,1.000000,0.000000
25%,52.944980,10.000000,0.000000,6.000000,63.000000,6.899414,72.000000,18.000000,36.195310,155.093750,2.399902,0.500000,0.899902,134.000000,7.379883,103.000000,14.000000,1175.000000
50%,65.015990,12.000000,0.000000,11.000000,77.000000,10.500000,100.000000,24.000000,36.695310,224.000000,2.899902,0.899902,1.199951,137.000000,7.419922,135.000000,23.000000,1964.000000
75%,74.135990,14.000000,1.000000,20.000000,107.000000,15.298828,120.000000,28.000000,38.195310,304.750000,3.599609,1.899902,1.899902,141.000000,7.469727,190.000000,43.000000,2980.000000
max,101.847960,31.000000,1.000000,343.000000,195.000000,128.781250,232.000000,90.000000,41.695310,869.375000,29.000000,63.000000,21.500000,181.000000,7.769531,1092.000000,192.000000,9000.000000


# Lagre dataen sånn kan bruke den til visualisering og se på unødvendige column, og ser mer nøye på dataen før forsetter

In [113]:
combined_df.to_csv('combined.csv') 

# Handling unnessecery columns